In [1]:
from pathlib import Path

import teehr

In [2]:
HEFS_DIR = "/home/sam/temp/hefs/Workspace/OHRFC/Baseline_validation/haksu.lee.ma/DEP_hindcasts1/Export/RTi_HEFS_Localflows/"

# EVAL_DIR = "/home/sam/temp/hefs/dwnn6_evaluation"

EVAL_DIR = "/data/playground/slamont/hefs/dwnn6_evaluation"   # remote

In [3]:
ev = teehr.Evaluation(dir_path=EVAL_DIR)
# ev.enable_logging()

constant_field_values = {
    "unit_name": "ft^3/s",
    "variable_name": "streamflow_hourly_inst"
}

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/15 15:03:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
ev.secondary_timeseries.load_fews_xml(
    in_path=HEFS_DIR,
    constant_field_values=constant_field_values
)

In [ ]:
# ev.secondary_timeseries.to_pandas()

In [ ]:
ev.spark.stop()

### Extras

#### Let's try grouping the cache files into bigger chunks

In [4]:
filelist = sorted(list(Path(EVAL_DIR, "cache", "loading", "secondary_timeseries").glob("*.parquet")))

In [5]:
len(filelist)

10500

In [6]:
filelist = [str(path) for path in filelist]

In [7]:
chunksize = 1000

chunks = [filelist[x:x+chunksize] for x in range(0, len(filelist), chunksize)]

In [8]:
len(chunks[0])

1000

In [9]:
len(chunks)

11

In [ ]:
# schema = self.schema_func().to_structtype()

In [10]:
# Create a temp cache dir
cache_dir = Path(EVAL_DIR, "cache", "loading", "secondary_timeseries_chunked")
cache_dir.mkdir(parents=True, exist_ok=True)

In [11]:
options = {
    "header": "true",
    "ignoreMissingFiles": "true"
}

In [13]:
%%time
for i, file_chunk in enumerate(chunks):

    df = ev.spark.read.format("parquet").options(**options).load(file_chunk)
    df = df.repartition(1)
    (
        df.
        write.
        format("parquet").
        mode("overwrite").
        save(str(cache_dir) + f"/chunk_{i}.parquet")
    )
    print(f"Chunk {i} done")
    # break


Chunk 0 done


Chunk 1 done


Chunk 2 done


Chunk 3 done


Chunk 4 done


...

Chunk 6 done


Chunk 7 done


Chunk 8 done


Chunk 9 done


Chunk 10 done
CPU times: user 450 ms, sys: 174 ms, total: 624 ms
Wall time: 19min 17s


In [ ]:
df.show()

In [ ]:
cache_dir